In [31]:
# header
import pandas as pd
from ast import literal_eval
from GenerateLexDir import literal_eval_col
from datetime import datetime
from FindBib import read_date

In [32]:
# initialize dfs
flexicon = pd.read_csv('flexiconPART.csv', index_col='entry_id', keep_default_na=False)
new_data = pd.read_csv('Flexport_2_1/Flexicon_NewDataPART.csv', index_col='entry_id', keep_default_na=False)
matches  = pd.read_csv('new_entries_matchesPART.csv', index_col='entry_id', keep_default_na=False)

In [33]:
# evaluate cols as native python types
literal_eval_col(flexicon, 'these_vars')
literal_eval_col(flexicon, 'variant_of')
literal_eval_col(flexicon, 'other_sources')

literal_eval_col(new_data, 'these_vars')
literal_eval_col(new_data, 'variant_of')

literal_eval_col(matches, 'matches')

In [34]:
delete_these = [x in ('delete from clean', 'delete new & clean') for x in matches['status']]
delete_these = matches[delete_these]
delete_these.shape

(55, 3)

In [35]:
# only include entries edited since recent field trip
cutoff = datetime(2019, 11, 3)
new = [read_date(t) > cutoff for t in new_data['date_modified']]
new_data = new_data[new]
new_data.shape

(280, 9)

In [36]:
# remove idcs of delete_these from flexicon
# but first, transfer any data in other_sources and these_vars to new_data

new_data.loc[:,'other_sources'] = [{} for row in new_data.iterrows()]

new_data.loc[:, 'these_vars'] = [x if x else {} for x in new_data['these_vars']]
        
for index, row in matches.iterrows():
    match_idcs = row['matches'].keys()
    fl_other_sources = [flexicon.at[i, 'other_sources'] for i in match_idcs]
    for x in fl_other_sources:
        if x:
            new_data.at[i, 'other_sources'].update(x)
    fl_these_vars =    [flexicon.at[i, 'these_vars']    for i in match_idcs]
    for x in fl_these_vars:
        if x:
            new_data.at[i, 'these_vars'].update(x)
    


In [37]:
# now we can drop all entries that have been marked for deletion from flexicon
print(flexicon.shape)
flexicon = flexicon.drop(match_idcs)
print(flexicon.shape)

(1339, 10)
(1337, 10)


In [38]:
# now we can drop all enries marked for deletion from new_data
delete_these = [x in ('delete new', 'delete clean & new') for x in matches['status']]
delete_these = matches[delete_these]
delete_these.shape

(8, 3)

In [39]:
# drop indices
for i in delete_these.index:
    new_data = new_data.drop(i)
new_data.shape

(272, 10)

In [40]:
# merge df's
merged_df = pd.concat([flexicon, new_data])
merged_df.shape

/home/mark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(1609, 11)

In [41]:
merged_df.to_csv('FlexiconMERGE.csv')